# Taxonomic classification: superkingdom prediction

In [1]:
from sequence_pairs import SampleList, make_cgr
from ete3 import NCBITaxa
import math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import confusion_matrix, pairwise, roc_auc_score, roc_curve
from sklearn.feature_selection import VarianceThreshold, chi2, SelectKBest, SelectPercentile
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, RepeatedKFold
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import simple_cgr_model

2023-09-14 16:55:33.648537: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
ncbi = NCBITaxa()

In [3]:
def checkfield(fullfield, key):
    parts = str(fullfield).split('|')
    for part in parts:
        if part.startswith(key+':'):
            uniprot_parts = part.split(':')
            upid = uniprot_parts[1]
            if '-' in upid:
                return upid[:upid.find('-')]
            else:
                return upid
    return None

def get_uniprot(row, protein_no):
    uniprot = checkfield(row[f'protein_xref_{protein_no}'],'uniprotkb')
    if uniprot is not None:
        return uniprot
    uniprot = checkfield(row[f'alternative_identifiers_{protein_no}'],'uniprotkb')
    if uniprot is not None:
        return uniprot
    uniprot = checkfield(row[f'protein_alias_{protein_no}'],'uniprotkb')
    if uniprot is not None:
        return uniprot
    return ''

def load_hpidb_table(table_filename):
    hpidb_data = pd.read_table(table_filename,encoding='latin1')
    hpidb_data=hpidb_data.rename(columns={'# protein_xref_1':'protein_xref_1'})
    ups_1 = []
    ups_2 = []
    for i in range(len(hpidb_data)):
        up_1 = get_uniprot(hpidb_data.iloc[i],1)
        up_2 = get_uniprot(hpidb_data.iloc[i],2)
        if up_1.find('(gene name)') != -1 or up_2.find('(gene name)') != -1:
            ups_1.append("")
            ups_2.append("")        
        else:
            ups_1.append(up_1)
            ups_2.append(up_2)
    hpidb_data['uniprot_1'] = ups_1
    hpidb_data['uniprot_2'] = ups_2
    hpidb_data = hpidb_data.query('uniprot_1 != "" and uniprot_2 != ""')
    return hpidb_data

def get_taxid(taxid_string):
    taxid = int(taxid_string.split('(')[0].replace('taxid:',''))
    return taxid

def get_species(taxid):
    rank = ncbi.get_rank([taxid]) 
    if len(rank) > 0:
        if rank[taxid] == 'species':
            species = taxid
        else:
            ranks = ncbi.get_rank(ncbi.get_lineage(taxid))
            potential_species = [r for r in ranks if ranks[r] == 'species']
            species = np.min(potential_species)
        return ncbi.get_taxid_translator([species])[species]
    else:
        return None

def species_mask_samples(sample_list):
    species_1 = {}
    species_2 = {}
    spec_masked_samples = []
    for sample_pair in sample_list:
        id_1 = sample_pair.seq_record_1.id
        id_2 = sample_pair.seq_record_2.id
        if id_1 in uniprot_to_taxid and id_2 in uniprot_to_taxid:
            taxid_1 = uniprot_to_taxid[id_1]
            taxid_2 = uniprot_to_taxid[id_2]
            sp1 = get_species(taxid_1)
            sp2 = get_species(taxid_2)
            if sp1 is not None and sp2 is not None:
                spec_masked_samples.append([str(sp1), str(sp2), sample_pair.is_ppi])
    return pd.DataFrame(spec_masked_samples, columns=['species_1','species_2','is_ppi'])

hpi_pred = load_hpidb_table('data/0009_superkingdom_classification/hpidb_interologs.mitab_plus.txt')
hpidb_data = load_hpidb_table('data/0009_superkingdom_classification/hpidb2.mitab.txt')

In [4]:
phibase_df = pd.read_csv('data/0009_superkingdom_classification/phi-base_current2.csv', encoding='latin-1')

/var/folders/_j/jxj9zb555rx6r9825t1lmz_r000fpf/T/ipykernel_89925/3282808939.py:1: DtypeWarning: Columns (22,25,26,29,30,37,45,46,47,50,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  phibase_df = pd.read_csv('data/0009_superkingdom_classification/phi-base_current2.csv', encoding='latin-1')


In [5]:
hpidb_data_combined = pd.concat((hpidb_data, hpi_pred))

In [6]:
uniprot_to_taxid = {}

for row in hpidb_data_combined.iloc:
    taxid_1 = get_taxid(row['protein_taxid_1'])
    taxid_2 = get_taxid(row['protein_taxid_2'])
    up_1 = row['uniprot_1']
    up_2 = row['uniprot_2']
    # If there are multiple different taxids for a given ID, take the lowest one (as the higher one will 
    # correspond to a specific strain, whereas both are definitely )
    if up_1 not in uniprot_to_taxid or (uniprot_to_taxid[up_1] != taxid_1 and taxid_1 < uniprot_to_taxid[up_1]):
        uniprot_to_taxid[up_1] = taxid_1
    if up_2 not in uniprot_to_taxid or (uniprot_to_taxid[up_2] != taxid_2 and taxid_2 < uniprot_to_taxid[up_2]):
        uniprot_to_taxid[up_2] = taxid_2

In [7]:
data = np.load('data/0000_datasets_as_used_in_paper/HPIDB/HPIDB3_4mers.npz',allow_pickle=True)
training=SampleList(data['training'])
validation=SampleList(data['validation'])
test=SampleList(data['test'])

## Species-masked data

In [8]:
def get_onehotencoding(all_df, train_df, valid_df, test_df):
    enc = OneHotEncoder()
    enc.fit(all_df)
    enc_train = enc.transform(train_df).toarray()
    enc_train_x = enc_train[:,:-2]
    enc_train_y = train_df['is_ppi']
    enc_valid = enc.transform(valid_df).toarray()
    enc_valid_x = enc_valid[:,:-2]
    enc_valid_y = valid_df['is_ppi']
    enc_test = enc.transform(test_df).toarray()
    enc_test_x = enc_test[:,:-2]
    enc_test_y = test_df['is_ppi']
    return (enc_train_x, enc_train_y, enc_valid_x, enc_valid_y, enc_test_x, enc_test_y)

In [9]:
# Mask sequence with species
spec_masked_df_train = species_mask_samples(training)
spec_masked_df_validation = species_mask_samples(validation)
spec_masked_df_test = species_mask_samples(test)
spec_masked_df = pd.concat((spec_masked_df_train,spec_masked_df_validation,spec_masked_df_test))

# one-hot encode the data for machine learning
spec_masked_df_enc_train_x, spec_masked_df_enc_train_y,spec_masked_df_enc_valid_x, spec_masked_df_enc_valid_y, spec_masked_df_enc_test_x, spec_masked_df_enc_test_y = get_onehotencoding(spec_masked_df,
spec_masked_df_train, spec_masked_df_validation ,spec_masked_df_test)

In [10]:
def plot_confusion_matrix(conf):
    # row = truth, column = prediction
    tn, fp = conf[0] # truth = negative
    fn, tp = conf[1] # truth = positive
    print('tp', tp, 'tn', tn, 'fp', fp, 'fn', fn)
    print('f1',tp/(tp+0.5*(fp+fn)))
    row_sums = conf.sum(axis=1)
    conf = conf / row_sums[:, np.newaxis]
    plt.imshow(conf, cmap='Blues')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.yticks([0,1])
    plt.xticks([0,1])
    midp = (np.amax(conf)+np.amin(conf))/2
    for i in range(2):
        for j in range(2):
            if conf[i][j] > midp:
                colour = 'w'
            else:
                colour = 'k'
            plt.text(j,i,'{:.2f}'.format(conf[i][j]),c=colour)
    plt.show()

# Build a k-nearest neighbour model and train using the data in the given data directory.
def build_knn_model(kneighbours, train_x, train_y, val_x, val_y, test_x, test_y, holdout):
    clf = KNeighborsClassifier(n_neighbors=kneighbours, n_jobs=1).fit(train_x, train_y)
    if holdout:
        ys = clf.predict(test_x)
        scores = clf.score(test_x, test_y)
        cm = confusion_matrix(test_y, ys)
        return scores, cm
    else:
        ys = clf.predict(val_x)
        scores = clf.score(val_x, val_y)
        cm = confusion_matrix(val_y, ys)
        return scores, cm

def run_knn(train_x, train_y, val_x, val_y, test_x, test_y):
    max_v = -math.inf
    max_i_dist = 0
    max_k = 5
    vs_dist = np.zeros((max_k,))
    for i in range(1, max_k):
        print(i)
        scores_temp, cm_temp = build_knn_model(i, train_x, train_y, val_x, val_y, test_x, test_y, False)
        vs_dist[i] = scores_temp
        if scores_temp > max_v:
            max_v = scores_temp
            max_i_dist = i
    print('Number of neighbours vs. classification score')
    plt.plot(vs_dist)
    plt.xlabel('k (number of neighbours)')
    plt.ylabel('Validation accuracy')
    plt.show()
    scores_dist, cm_dist = build_knn_model(max_i_dist, train_x, train_y, val_x, val_y, test_x, test_y, True)
    print('Score:',scores_dist)
    plot_confusion_matrix(cm_dist)

In [11]:
run_knn(spec_masked_df_enc_train_x, spec_masked_df_enc_train_y,
                                           spec_masked_df_enc_valid_x, spec_masked_df_enc_valid_y,
                                           spec_masked_df_enc_test_x, spec_masked_df_enc_test_y)

## Build superkingdom data

CGRs for sequences, labelled by superkingdom

In [11]:
def get_specific_rank(taxid, rankname):
    rank = ncbi.get_rank([taxid]) 
    if len(rank)>0:
        if rank[taxid] == rankname:
            species = taxid
        else:
            ranks = ncbi.get_rank(ncbi.get_lineage(taxid))
            potential_species = [r for r in ranks if ranks[r] == rankname]
            if len(potential_species) == 0:
                return None
            species = np.min(potential_species)
        return ncbi.get_taxid_translator([species])[species]
    else:
        return None


def remake_data(dataset, rank, k=4, debug=False):
    new_dataset_x = []
    new_dataset_y = []
    for sample_pair in dataset:
        id_1 = sample_pair.seq_record_1.id
        id_2 = sample_pair.seq_record_2.id
        if id_1 in uniprot_to_taxid and id_2 in uniprot_to_taxid:
            taxid_1 = uniprot_to_taxid[id_1]
            taxid_2 = uniprot_to_taxid[id_2]
            rank_1 = get_specific_rank(taxid_1, rank)
            rank_2 = get_specific_rank(taxid_2, rank)
            if rank_1 is not None:
                cgr_1 = make_cgr(str(sample_pair.seq_record_1.seq), k, True)
                new_dataset_x.append(cgr_1)
                new_dataset_y.append(rank_1)
            if rank_2 is not None:
                cgr_2 = make_cgr(str(sample_pair.seq_record_2.seq), k, True)
                cgr_2 = cgr_2 / np.amax(cgr_2)
                new_dataset_x.append(cgr_2)
                new_dataset_y.append(rank_2)
            if debug and len(new_dataset_y)>10:
                break
    return np.array(new_dataset_x)[:,:,:,None], new_dataset_y

def encode_data(rank, k, debug=False):
    spec_labelled_data_train_x, spec_labelled_data_train_y = remake_data(training, rank, k, debug)
    enc = OneHotEncoder(handle_unknown='ignore')
    enc_train = enc.fit_transform(np.array(spec_labelled_data_train_y).reshape(-1, 1)).toarray()
    spec_labelled_data_validation_x, spec_labelled_data_validation_y = remake_data(validation, rank, k, debug)
    spec_labelled_data_test_x, spec_labelled_data_test_y = remake_data(test, rank, k, debug)
    enc_validation = enc.transform(np.array(spec_labelled_data_validation_y).reshape(-1, 1)).toarray()
    enc_test = enc.transform(np.array(spec_labelled_data_test_y).reshape(-1, 1)).toarray()
    val_inds = np.argwhere(np.sum(enc_validation,axis=1,dtype=int))[:,0]
    test_inds = np.argwhere(np.sum(enc_test,axis=1,dtype=int))[:,0]
    enc_validation=enc_validation[val_inds,:]
    spec_labelled_data_validation_x=spec_labelled_data_validation_x[val_inds,:,:,:]
    enc_test=enc_test[test_inds,:]
    spec_labelled_data_test_x=spec_labelled_data_test_x[test_inds,:,:,:]
    species_list = np.unique(spec_labelled_data_train_y)
    class_weight_list = compute_class_weight(class_weight='balanced',classes=species_list,
                                             y=spec_labelled_data_train_y)
    class_weights = {i: class_weight_list[i] for i in range(len(np.unique(spec_labelled_data_train_y)))}
    return spec_labelled_data_train_x, enc_train, spec_labelled_data_validation_x, \
        enc_validation, spec_labelled_data_test_x, enc_test, species_list, class_weights

In [12]:
train_x, train_y, val_x, val_y, test_x, test_y, species_list, class_weights = encode_data('superkingdom',4)
np.savez('data/0009_superkingdom_classification/superkingdom_4mers.npz',train_x=train_x,train_y=train_y,
        val_x=val_x,val_y=val_y,
         test_x=test_x,test_y=test_y,
        species_list=species_list,class_weights=class_weights)

## Run parameter search

Ran elsewhere for GPU access

## Holdout test

In [2]:
# Load in results from GPU
results_df=pd.read_csv(f'data/0009_superkingdom_classification/superkingdom_out.csv')

In [3]:
results_df

,Unnamed: 0,conv_blocks,num_filters,num_conv_layers_per_block,kernel_width,dense_layer_depth,dense_layers,conv_batch_normalisation,dense_batch_normalisation,rate_of_dropout,...,epoch,max_val_acc,loss,acc,categorical_accuracy,categorical_crossentropy,val_loss,val_acc,val_categorical_accuracy,val_categorical_crossentropy
0,0,4,1024,2,3,64,3,1,1,0.3,...,5,0.666540,0.667945,0.683215,0.683215,0.662116,1.166011,0.501906,0.501906,1.166011
1,1,4,1024,2,3,64,3,1,1,0.3,...,23,0.919588,0.121198,0.950609,0.950609,0.144717,0.267529,0.918445,0.918445,0.267529
2,2,4,1024,2,3,64,3,1,1,0.3,...,17,0.924543,0.204820,0.915693,0.915693,0.230340,0.400331,0.873476,0.873476,0.400331
3,3,4,1024,2,3,64,3,1,1,0.3,...,18,0.922256,0.214259,0.910742,0.910742,0.250642,0.441605,0.863567,0.863567,0.441605
4,4,4,1024,2,3,64,3,1,1,0.3,...,19,0.929878,0.163170,0.932326,0.932326,0.195568,0.239698,0.929878,0.929878,0.239698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,1,64,2,3,24,0,0,0,0.5,...,97,0.825457,0.629248,0.776917,0.776917,0.593193,0.552271,0.816311,0.816311,0.552271
496,496,1,64,2,3,24,0,0,0,0.5,...,130,0.828506,0.571401,0.791519,0.791519,0.551607,0.517950,0.818979,0.818979,0.517950
497,497,1,64,2,3,24,0,0,0,0.5,...,96,0.833841,0.604332,0.782377,0.782377,0.577377,0.535600,0.815168,0.815168,0.535600
498,498,1,64,2,3,24,0,0,0,0.5,...,105,0.829268,0.602025,0.786821,0.786821,0.568174,0.531216,0.820503,0.820503,0.531216


In [4]:
means=results_df.groupby('seed_i',as_index=False).mean()
seed_i=means.sort_values(by='val_categorical_accuracy',ascending=False).iloc[0]['seed_i']
results_df.query(f'seed_i=={seed_i}')

,Unnamed: 0,conv_blocks,num_filters,num_conv_layers_per_block,kernel_width,dense_layer_depth,dense_layers,conv_batch_normalisation,dense_batch_normalisation,rate_of_dropout,...,epoch,max_val_acc,loss,acc,categorical_accuracy,categorical_crossentropy,val_loss,val_acc,val_categorical_accuracy,val_categorical_crossentropy
115,115,3,512,1,5,512,0,1,1,0.7,...,19,0.949314,0.062419,0.971813,0.971813,0.077183,0.255558,0.942454,0.942454,0.255558
116,116,3,512,1,5,512,0,1,1,0.7,...,15,0.946646,0.100834,0.956831,0.956831,0.123142,0.183999,0.946646,0.946646,0.183999
117,117,3,512,1,5,512,0,1,1,0.7,...,18,0.948933,0.063076,0.973083,0.973083,0.079529,0.302936,0.948933,0.948933,0.302936
118,118,3,512,1,5,512,0,1,1,0.7,...,16,0.937119,0.118746,0.954545,0.954545,0.132777,0.462827,0.897104,0.897104,0.462827
119,119,3,512,1,5,512,0,1,1,0.7,...,17,0.948552,0.092838,0.965846,0.965846,0.104901,0.202754,0.946646,0.946646,0.202754


In [5]:
best_params={}
for key in simple_cgr_model.possible_params:
    if key in ['rate_of_dropout','rate_of_conv_dropout','learning_rate']:
        best_params[key]=means.sort_values('val_categorical_accuracy').iloc[-1][key]
    else:
        best_params[key]=int(means.sort_values('val_categorical_accuracy').iloc[-1][key])

In [6]:
best_params

{'conv_blocks': 3,
 'num_filters': 512,
 'num_conv_layers_per_block': 1,
 'kernel_width': 5,
 'dense_layer_depth': 512,
 'dense_layers': 0,
 'conv_batch_normalisation': 1,
 'dense_batch_normalisation': 1,
 'rate_of_dropout': 0.7,
 'rate_of_conv_dropout': 0.3,
 'conv_use_bias': 0,
 'dense_use_bias': 1,
 'learning_rate': 0.001,
 'optimizer': 0}

In [7]:
data = np.load('data/0000_datasets_as_used_in_paper/Taxonomy/superkingdom_4mers.npz',allow_pickle=True)
train_x = data['train_x']
train_y = data['train_y']
val_x = data['val_x']
val_y = data['val_y']
test_x = data['test_x']
test_y = data['test_y']
species_list = data['species_list']
class_weights = data['class_weights'].item()

In [ ]:
simple_cgr_model.test_params(best_params,species_list,0,
                             train_x, train_y, val_x, val_y, test_x, test_y, class_weights)